In [ ]:
!nvidia-smi

Thu Aug  3 22:21:11 2023       

+-----------------------------------------------------------------------------+

| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |

|-------------------------------+----------------------+----------------------+

| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |

| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |

|                               |                      |               MIG M. |

|===============================+======================+======================|

|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |

| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |

|                               |                      |                  N/A |

+-------------------------------+----------------------+----------------------+

                                                                            

In [4]:
!pip install torch
!pip install bitsandbytes
!pip install sentencepiece
!pip install accelerate@git+https://github.com/huggingface/accelerate.git
!pip install transformers@git+https://github.com/huggingface/transformers.git
!pip install peft@git+https://github.com/huggingface/peft.git
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.7 MB/s eta 0:00:0000:0100:01
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-vheyku5w/accelerate_1ef1638a693243dbb5f129e0879b01a4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-vheyku5w/accelerate_1ef1638a693243dbb5f129e0879b01a4
  Resolved https://github.com/huggingface/accelerate.git to commit 30eff605df5db9ba20289a0c7471e4d8547a5a8f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.22.0.dev0-py3-none-any.whl size=249332 sha256=fce91f2860ba2836a4737a76a405ced74134f9b529e94f3ffc5f6292d0ea31a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-smurji_k/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found 

In [ ]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [8]:
base_model_name_or_path = "VMware/xgen-7b-8k-open-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, use_fast=False, trust_remote_code = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name_or_path, torch_dtype=torch.bfloat16, load_in_4bit=True, device_map='sequential')

A new version of the following files was downloaded from https://huggingface.co/VMware/xgen-7b-8k-open-instruct:
- tokenization_xgen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Load the LoRA adapter

#lora_model_name_or_path = "22h/cabrita-lora-v0-1"
#model = PeftModel.from_pretrained(model, lora_model_name_or_path)

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

In [10]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Output:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Output:
"""

In [13]:
def instruct(
    instruction ,
    temperature=0.3,
    top_p=0.9,
    max_new_tokens=200,
    input=None,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    do_sample = temperature > 0.0

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
            **kwargs,
          ),
        return_dict_in_generate=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    splitted_generated_text = generated_text.split('\n')
    first_generated_text = splitted_generated_text[0]

    n = 1
    while first_generated_text == "":
        if splitted_generated_text[n]:
            first_generated_text = splitted_generated_text[n]
        else:
            break

    return first_generated_text

In [14]:
traducao = instruct("Translate the following text from english to portuguese.",input="He's never done anything.")

print(traducao)

Não fez nada nunca.


In [15]:
import pandas as pd

model_code = 'xgen'

try:
  opus_100_pt_validation = pd.read_parquet('/notebooks/opus_100_pt_validation_' + model_code + '.parquet')
except:
  opus_100_pt_validation = pd.read_parquet('https://huggingface.co/api/datasets/opus100/parquet/en-pt/validation/0.parquet')

In [ ]:
max_string = ""

for json in opus_100_pt_validation.translation:

    string = json['en']
    if len(string) > len(max_string):
        max_string = string

    string = json['pt']
    if len(string) > len(max_string):
        max_string = string

print("The biggest string is: " + max_string)

tokens = tokenizer.tokenize(max_string)
num_tokens = len(tokens)

print("The maximum tokens number is: " + str(num_tokens))

The biggest string is: having regard to the Annual Report of the Court of Auditors on the implementation of the budget concerning the financial year 2011, together with the institutions’ replies [3], and to the Court of Auditors’ special reports,having regard to the statement of assurance [4] as to the reliability of the accounts and the legality and regularity of the underlying transactions provided by the Court of Auditors for the financial year 2011 pursuant to Article 287 of the Treaty on the Functioning of the European Union,having regard to the Council’s recommendation of 12 February 2013 on discharge to be given to the Commission in respect of the implementation of the budget for the financial year 2011 (05752/2013 – C7-0038/2013),

The maximum tokens number is: 189


In [ ]:
dataset = opus_100_pt_validation.translation

row_number = len(dataset)
current_row = 0

for json in opus_100_pt_validation.translation:
    current_row=current_row+1

    if model_code in json and json[model_code]:
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução existente: " + json[model_code] )
    else:
        originalString = json['en']
        traducao = instruct("Translate the following text from english to portuguese.",input=originalString)
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução: " + traducao)
        json[model_code]=traducao
        opus_100_pt_validation.to_parquet('./opus_100_pt_validation_' + model_code + '.parquet')


1/2000 - Tradução: Não fez nada nunca.
2/2000 - Tradução: Chute de 50% a 50%?
3/2000 - Tradução: Yeah, eu sou deixando você se ajustar até você seja bom.
4/2000 - Tradução: Isso foi você mesmo?
5/2000 - Tradução: E então você vai lá!
6/2000 - Tradução: Porque você não está lavando.
7/2000 - Tradução: Pergunta, seu telefone tocou a voz de Parker?
8/2000 - Tradução: Porque eu sei que você vai me perguntar sobre como eu disse que eu era um dos autores deste livro. Eu não sei m caminho da minha fonte, mas eu sei que eu disse a verdade.
9/2000 - Tradução: Se você amassei, teus braços teriam me dado a mão.
10/2000 - Tradução: - Possíveis são nossos dois suspeitos.
11/2000 - Tradução: - Adicionamos a portas...
12/2000 - Tradução: A partir do orçamento operacional, destina-se a subsídio de juros para atração de um subvenção.
13/2000 - Tradução: Nenhum reis. Assim, Mikey Molloy fingiu um de seus ataques como um pretexto para que eu pudesse entrar quando ninguém estava olhando.
14/2000 - Traduçã

In [ ]:
opus_100_pt_validation.to_parquet('./opus_100_pt_validation_' + model_code + '.parquet')

In [ ]:

saved_parquet = pd.read_parquet('./opus_100_pt_validation_' + model_code + '.parquet')
print(saved_parquet.translation[200])